In [ ]:
# python3 /home/rnb/Projects/rnb-planning/src/pkg/planning/filtering/lattice_model/shared_lattice_predictor.py --rtype panda

# import SharedArray as sa
# ROBOT_TYPE_NAME = 'indy7gripper'
# sa.delete("shm://{}.prepared".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.grasp_img".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.arm_img".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.rh_vals".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.result".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.query_in".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.response_out".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.query_quit".format(ROBOT_TYPE_NAME))

In [1]:
from __future__ import print_function
import os
RNB_PLANNING_DIR = os.environ["RNB_PLANNING_DIR"]
os.chdir(os.path.join(RNB_PLANNING_DIR, 'src'))
from pkg.controller.combined_robot import *
from pkg.utils.utils import get_now, try_mkdir
import subprocess

DATA_PATH = os.path.join(RNB_PLANNING_DIR, "data")
try_mkdir(DATA_PATH)

DAT_ROOT = "stowing-deep"
RES_ROOT = "stowing-deep-result"

TEST_DATA_PATH = os.path.join(DATA_PATH, DAT_ROOT)
try_mkdir(TEST_DATA_PATH)

TEST_RESULT_PATH = os.path.join(DATA_PATH, RES_ROOT)
try_mkdir(TEST_RESULT_PATH)

HEADS = ["plan_time", "length", "MP_count", "failed_MPs", "IK_count", "failed_IKs", "success"]

# obj1

In [2]:
RTYPE = "indy7gripper"
DAT_DIR = '20210829-050034'
FILE_OPTION = "obj_1"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210705-182244'
# FILE_OPTION = "obj_1"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
# CNAME_LIST = ['Full'] #,'Pairwise']
GRASP_SAMPLE = 100
VISUALIZE = False
PLAY_RESULT = False
SHOW_STATE = False

TIMEOUT_MOTION = 5
MAX_TIME = 100
MAX_ITER = 100


### test

In [3]:
I_START = 0
I_END = 10
N_DAT = I_END-I_START

In [4]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(N_DAT):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid+I_START),
                              '--cname', cname,
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid+1, N_DAT), 1)))

------- 3.9     3.1     3.0     104.7   -------
========== 1 / 10 (eta: 114.7 / 1147.4 s) =========
------- 2.3     2.6     3.1     3.5     -------
========== 2 / 10 (eta: 126.2 / 631.1 s) =========


KeyboardInterrupt: 

# obj3

In [ ]:
RTYPE = "indy7gripper"
DAT_DIR = '20210829-050034'
FILE_OPTION = "obj_3"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210705-182244'
# FILE_OPTION = "obj_3"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

# CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
# GRASP_SAMPLE = 100
# VISUALIZE = True
# PLAY_RESULT = True
# TIMEOUT_MOTION = 5
# MAX_TIME = 100
# SHOW_STATE = False


In [ ]:
I_START=0
I_END = 10
N_DAT = I_END-I_START

### test

In [ ]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(N_DAT):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid+I_START),
                              '--cname', cname,
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid+1, N_DAT), 1)))

# obj3_pole

In [ ]:
RTYPE = "indy7gripper"
DAT_DIR = '20210829-050034'
# DAT_DIR = '20210702-152658'
FILE_OPTION = "obj_3_pole"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210705-182244'
# FILE_OPTION = "obj_3_pole"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

# CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
# GRASP_SAMPLE = 100
# VISUALIZE = True
# PLAY_RESULT = True
# TIMEOUT_MOTION = 5
# MAX_TIME = 100
# SHOW_STATE = False


In [ ]:
I_START=0
I_END = 10
N_DAT = I_END-I_START

### test

In [ ]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')
gtimer.tic("full_loop")
for fid in range(N_DAT):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid+I_START),
                              '--cname', cname,
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid+1, N_DAT), 1)))

# obj3_hard

In [ ]:
RTYPE = "indy7gripper"
DAT_DIR = '20210829-050034'
# DAT_DIR = '20210702-152658'
FILE_OPTION = "obj_3_hard"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210705-182244'
# FILE_OPTION = "obj_3_pole"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

# CNAME_LIST = ['ToolReach','Full'] #,'Pairwise']
# # CNAME_LIST = ['Full'] #,'Pairwise']
# GRASP_SAMPLE = 100
# VISUALIZE = False
# PLAY_RESULT = False
# SHOW_STATE = False

# TIMEOUT_MOTION = 5
# MAX_TIME = 100
# MAX_ITER = 100

In [ ]:
I_START=0
I_END = 10
N_DAT = I_END-I_START

### test

In [ ]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')
gtimer.tic("full_loop")
for fid in range(N_DAT):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid+I_START),
                              '--cname', cname,
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid+1, N_DAT), 1)))